In [1]:
import zipfile

In [2]:
with zipfile.ZipFile('brain_dataset.zip', 'r') as zip_ref:
    zip_ref.extractall('extracted_data')

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
import timm
import os


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


cuda


In [7]:
DATA_DIR = "/content/brain_dataset"

TRAIN_DIR = os.path.join(DATA_DIR, "Training")
TEST_DIR  = os.path.join(DATA_DIR, "Testing")

BATCH_SIZE = 8
IMG_SIZE = 224


In [8]:
train_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])


In [9]:
train_dataset = datasets.ImageFolder(
    root=TRAIN_DIR,
    transform=train_transform
)

test_dataset = datasets.ImageFolder(
    root=TEST_DIR,
    transform=test_transform
)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

NUM_CLASSES = len(train_dataset.classes)
print("Classes:", train_dataset.classes)


Classes: ['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor']


In [10]:
class HybridSwinCNN(nn.Module):
    def __init__(self, num_classes):
        super(HybridSwinCNN, self).__init__()

        self.swin = timm.create_model(
            'swin_tiny_patch4_window7_224',
            pretrained=True,
            num_classes=0
        )

        self.cnn = models.resnet18(pretrained=True)
        self.cnn.fc = nn.Identity()

        swin_features = self.swin.num_features
        cnn_features = 512

        self.classifier = nn.Sequential(
            nn.Linear(swin_features + cnn_features, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        swin_feat = self.swin(x)
        cnn_feat = self.cnn(x)
        fused = torch.cat((swin_feat, cnn_feat), dim=1)
        return self.classifier(fused)


In [11]:
model = HybridSwinCNN(NUM_CLASSES).to(device)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/114M [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 126MB/s]


In [12]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-4)


In [13]:
def train_epoch(model, loader):
    model.train()
    loss_sum, correct, total = 0, 0, 0

    for imgs, labels in loader:
        imgs, labels = imgs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        loss_sum += loss.item()
        correct += (outputs.argmax(1) == labels).sum().item()
        total += labels.size(0)

    return loss_sum / len(loader), correct / total


In [14]:
def test_epoch(model, loader):
    model.eval()
    loss_sum, correct, total = 0, 0, 0

    with torch.no_grad():
        for imgs, labels in loader:
            imgs, labels = imgs.to(device), labels.to(device)

            outputs = model(imgs)
            loss = criterion(outputs, labels)

            loss_sum += loss.item()
            correct += (outputs.argmax(1) == labels).sum().item()
            total += labels.size(0)

    return loss_sum / len(loader), correct / total


In [15]:
EPOCHS = 15

for epoch in range(EPOCHS):
    train_loss, train_acc = train_epoch(model, train_loader)
    test_loss, test_acc = test_epoch(model, test_loader)

    print(f"Epoch [{epoch+1}/{EPOCHS}] "
          f"Train Acc: {train_acc:.4f} | "
          f"Test Acc: {test_acc:.4f}")


Epoch [1/15] Train Acc: 0.8091 | Test Acc: 0.6345
Epoch [2/15] Train Acc: 0.9220 | Test Acc: 0.6853
Epoch [3/15] Train Acc: 0.9432 | Test Acc: 0.7741
Epoch [4/15] Train Acc: 0.9627 | Test Acc: 0.7817
Epoch [5/15] Train Acc: 0.9571 | Test Acc: 0.7716
Epoch [6/15] Train Acc: 0.9791 | Test Acc: 0.7563
Epoch [7/15] Train Acc: 0.9756 | Test Acc: 0.7868
Epoch [8/15] Train Acc: 0.9714 | Test Acc: 0.7437
Epoch [9/15] Train Acc: 0.9815 | Test Acc: 0.7589
Epoch [10/15] Train Acc: 0.9819 | Test Acc: 0.7893
Epoch [11/15] Train Acc: 0.9805 | Test Acc: 0.7843
Epoch [12/15] Train Acc: 0.9857 | Test Acc: 0.7462
Epoch [13/15] Train Acc: 0.9791 | Test Acc: 0.7310
Epoch [14/15] Train Acc: 0.9850 | Test Acc: 0.7665
Epoch [15/15] Train Acc: 0.9836 | Test Acc: 0.7792


In [22]:
torch.save(model.state_dict(), "/content/hybrid_swin_cnn.pth")


In [17]:
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
import pandas as pd


In [18]:
model.eval()

all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        preds = torch.argmax(outputs, dim=1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())


In [19]:
cm = confusion_matrix(all_labels, all_preds)
class_names = train_dataset.classes

cm_df = pd.DataFrame(cm, index=class_names, columns=class_names)
print("Confusion Matrix:")
cm_df


Confusion Matrix:


,glioma_tumor,meningioma_tumor,no_tumor,pituitary_tumor
glioma_tumor,25,54,21,0
meningioma_tumor,0,115,0,0
no_tumor,0,0,105,0
pituitary_tumor,0,6,6,62


In [20]:
report = classification_report(
    all_labels,
    all_preds,
    target_names=class_names,
    digits=4
)

print("Classification Report:\n")
print(report)


Classification Report:

                  precision    recall  f1-score   support

    glioma_tumor     1.0000    0.2500    0.4000       100
meningioma_tumor     0.6571    1.0000    0.7931       115
        no_tumor     0.7955    1.0000    0.8861       105
 pituitary_tumor     1.0000    0.8378    0.9118        74

        accuracy                         0.7792       394
       macro avg     0.8631    0.7720    0.7477       394
    weighted avg     0.8454    0.7792    0.7404       394



In [21]:
report_dict = classification_report(
    all_labels,
    all_preds,
    target_names=class_names,
    output_dict=True
)

metrics_df = pd.DataFrame(report_dict).transpose()
metrics_df


,precision,recall,f1-score,support
glioma_tumor,1.000000,0.250000,0.400000,100.000000
meningioma_tumor,0.657143,1.000000,0.793103,115.000000
no_tumor,0.795455,1.000000,0.886076,105.000000
pituitary_tumor,1.000000,0.837838,0.911765,74.000000
accuracy,0.779188,0.779188,0.779188,0.779188
macro avg,0.863149,0.771959,0.747736,394.000000
weighted avg,0.845417,0.779188,0.740395,394.000000


from matplotlib import pyplot as plt
_df_0['precision'].plot(kind='hist', bins=20, title='precision')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['recall'].plot(kind='hist', bins=20, title='recall')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2['f1-score'].plot(kind='hist', bins=20, title='f1-score')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_3['support'].plot(kind='hist', bins=20, title='support')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_13['index'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_13, x='recall', y='index', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_14['index'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_14, x='f1-score', y='index', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_15['index'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_15, x='support', y='index', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)